# Test de plusieurs modèles avec pondération sur Houses prices
----

In [1]:
%run house_prices_prepare_data.ipynb

(1460, 81)
(1459, 80)


/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')
Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'Kitch

<Figure size 640x480 with 1 Axes>

<Figure size 1200x900 with 2 Axes>

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

corr : 0.36
corr : 0.38
corr : 0.26
corr : 0.34
corr : 0.39
corr : 0.39
corr : 0.66
corr : 0.67
corr : 0.69


/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


corr : 0.34
corr : 0.35
corr : 0.33
corr : 0.36
(1444, 80)
Supression de 16 outliers
(1444, 80)
(1459, 79)
(1444, 79)
(1444, 67)
(1459, 79)
(1459, 67)
46 features quantitatives
21 features qualitatives
Stored 'quan' (Index)
(1444, 187)
(1459, 187)


On recupere les id du fichier Kaggle pour la soumission

In [2]:
test_ID

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

Train et Test avec traitement des données , log1p sur SalePrice , robustScaler 

In [3]:
print(train.shape)
print(test.shape)

(1444, 187)
(1459, 187)


In [4]:
train.head()

,LotFrontage,LotArea,Street,Alley,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_class13
0,-0.25,-0.252090,0.0,0.0,0.0,0.5,0.0,0.648649,0.243243,1.200613,...,0,0,0,1,0,0,0,1,0,0
1,0.50,0.037091,0.0,0.0,0.0,0.0,3.0,0.064865,-0.486486,0.000000,...,0,0,0,1,0,0,0,1,0,0
2,-0.10,0.452002,0.0,0.0,0.0,0.5,0.0,0.605405,0.216216,0.992343,...,0,0,0,1,0,0,0,1,0,0
3,-0.50,0.024518,0.0,0.0,0.0,0.5,0.0,-1.254054,-0.648649,0.000000,...,0,0,0,1,0,0,0,0,0,0
4,0.70,1.208902,0.0,0.0,0.0,1.0,0.0,0.583784,0.162162,2.143951,...,0,0,0,1,0,0,0,1,0,0


In [5]:
test.head()

,LotFrontage,LotArea,Street,Alley,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MSSubClass_class13
0,0.50,0.545546,0.0,0.0,0.0,-0.5,1.0,-0.259459,-0.891892,0.000000,...,0,0,0,1,0,0,0,1,0,0
1,0.55,1.210662,0.0,0.0,0.0,0.0,1.0,-0.324324,-0.972973,0.661562,...,0,0,0,1,0,0,0,1,0,0
2,0.20,1.100773,0.0,0.0,0.0,-0.5,0.0,0.518919,0.108108,0.000000,...,0,0,0,1,0,0,0,1,0,0
3,0.40,0.132143,0.0,0.0,0.0,0.0,1.0,0.540541,0.108108,0.122511,...,0,0,0,1,0,0,0,1,0,0
4,-1.35,-1.118376,0.0,0.0,0.0,1.0,0.0,0.410811,-0.054054,0.000000,...,0,0,0,1,0,0,0,1,0,0


In [6]:
y_train

array([12.24769912, 12.10901644, 12.31717117, ..., 12.25486757,
       12.49313327, 11.86446927])

In [7]:
X_train = train.values

In [8]:
X_test = test.values

#### différents modeles

In [9]:
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import numpy as np

In [10]:
# Cross-validation with k-folds
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [11]:
ridge = Ridge()
model_xgb = xgb.XGBRegressor()
model_lgb = lgb.LGBMRegressor()

In [12]:
score = rmsle_cv(ridge)
print("Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
score = rmsle_cv(model_lgb)
print("Model LGB score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ridge score: 0.1161 (0.0048)

[18:15:24] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:25] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:25] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:26] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Xgboost score: 0.1229 (0.0038)

Model LG

#### Ridge

In [13]:
ridge = Ridge()

param_grid = [
    {'solver' : ['svd'], 'alpha':[15]}
    ]
grid_search = GridSearchCV(ridge,param_grid,cv=5, scoring='neg_mean_squared_error' ,return_train_score=True)

grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'solver': ['svd'], 'alpha': [15]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [14]:
grid_search.best_estimator_

Ridge(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='svd', tol=0.001)

In [15]:
y_ridge = np.expm1(grid_search.best_estimator_.predict(X_test))

### Xgb

In [16]:
model_xgb = XGBRegressor()

param_grid = [
    {'n_estimators': [371], 'max_depth':[2]}
    ]
grid_search = GridSearchCV(model_xgb,param_grid,cv=5, scoring='neg_mean_squared_error' ,return_train_score=True)

grid_search.fit(X_train,y_train)

[18:15:28] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:29] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:31] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:32] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:33] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:15:34] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [371], 'max_depth': [2]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [17]:
grid_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=371,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [18]:
y_xgb = np.expm1(grid_search.best_estimator_.predict(X_test))

### LGB Regressor

In [19]:
model_lgb = lgb.LGBMRegressor()

param_grid = [
    {'n_estimators': [2148], 'max_depth':[1]}
    ]
grid_search = GridSearchCV(model_lgb,param_grid,cv=5, scoring='neg_mean_squared_error' ,return_train_score=True)

grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [2148], 'max_depth': [1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [20]:
grid_search.best_estimator_

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=2148, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [21]:
y_lgb = np.expm1(grid_search.best_estimator_.predict(X_test))

#### Prédictions

In [22]:
y_val_kaggle = y_ridge * 0.6 + y_xgb * 0.2 + y_lgb*0.2

In [23]:
y_val_kaggle

array([115215.40384073, 152017.27656284, 178729.22716705, ...,
       166124.38782941, 118157.64216607, 211308.43677743])

In [24]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = y_val_kaggle
sub.to_csv('xgb_ridge_lgb.csv',index=False)